In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from copy import copy
from decimal import Decimal

# Problem 1

In [2]:
class UpAndOutPut:

    def __init__(self, K, T, barrier, observationinterval):
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

In [3]:
hw1contract = UpAndOutPut(K=95, T=0.25, barrier=114, observationinterval=0.02)

In [4]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [5]:
hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [18]:
class TreeEngine:

    def __init__(self, N):
        self.N = N

    def price_upandout(self, dynamics, contract):

        deltat = contract.T / self.N
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5)

        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)
        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            raise ValueError("This value of N fails to place the observation dates in the tree.")

        #slide 45 lec 1 
        nu = dynamics.rGrow - 0.5*(dynamics.sigma**2)       
        Pu = 0.5*((dynamics.sigma**2 * deltat + nu**2 * deltat**2)/(deltax**2)+(nu*deltat)/(deltax))       
        Pd = 0.5*((dynamics.sigma**2 * deltat + nu**2 * deltat**2)/(deltax**2)-(nu*deltat)/(deltax))      
        Pm = 1 - Pu - Pd      

        optionprice = np.maximum(contract.K-Sgrid,0)  

        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat:

            optionprice = np.exp(-dynamics.r*deltat) * optionprice[:-2] * Pu + optionprice[1:-1] * Pm + optionprice[2:] * Pd
            if abs(t/contract.observationinterval-round(t/contract.observationinterval)) < 1e-8:
                Sgrid_ob = Sgrid[1:-1] 
                optionprice = np.where(Sgrid_ob >= contract.barrier, 0, optionprice)        
            Sgrid = Sgrid[1:-1]
        return optionprice[0]



In [19]:
hw1tree=TreeEngine(N=100000)

hw1tree.price_upandout(hw1dynamics, hw1contract)

5.301058572410862

### (b)

From part a, we know that the price of up-and-out put is roughly 5.3010. If we write down the formula for the up-and-out put as well as up-and-in put: 
$$\text{up-and-out P} = (K-S_T)^+1_{max t \in \tau S_t < H} \\ \text{up-and-in P} = (K-S_T)^+1_{max t \in \tau S_t \geq H} \\ \text{European P} = (K-S_T)^+ \\$$

European P = up-and-out P + up-and-in P

According to BS Formula, we can get: 
$$\text{European p} = e^{-rT}KN(-d_2)-S(0)N(-d_1) = 95*N(-0.156)-100*N(-0.356) = 5.5195\\ \text{up-and-in put(0)} = 5.5195 - 5.3010 = 0.2185$$

### (c.1)

The option price should be lower, since the monitoring period is longer, there is a higher chance that teh put option will gain 0 at the end. Therefore, considering the trinomial tree model, more nodes might become 0, which lead to a lower value.

### (c.2)

Setup:
$$
p(K_p = 95, T = 0.25, \sigma = 0.4, r = 0); p(0) = 5.5195 \\
c(K_c = 136.8, T = 0.25, \sigma = 0.4, r = 0); c(0) = 0.586\\
\text{At barrier case: } C_{114} = N(d_1)*114-136.8*N(d_2), P_{114} = 95*N(-d_2) - N(-d_1)*114  \\
ln(114/95) = 0.18232155679 = -ln(114/136.8) = a \\
d_1^{call} = \frac{1}{0.4\sqrt{T-t}}[a +\frac{0.4^2}{2}*(T-t)] \\
d_2^{call} = d_1^{call} - 0.04\sqrt{T-t}\\
-d_1^{put} =  \frac{1}{0.4\sqrt{T-t}}[a -\frac{0.4^2}{2}*(T-t)] = d_2^{call} \\
-d_2^{put} = \frac{1}{0.4\sqrt{T-t}}[a -\frac{0.4^2}{2}*(T-t)] + 0.04\sqrt{T-t}= d_1^{call} \\

C(t) = N(d_1^{call}) * 114 - N(d_2^{call}) * 136.8 \\
P(t) = N(d_1^{call}) * 95 - N(d_2^{call}) * 114 \\ 
P(t) - \alpha C(t) =  (-114*\alpha + 95)N(d_1^{call}) +(136.8*\alpha - 114)N(d_2^{call}) = 0\\
\alpha = 0.833333 \\
p(cts) = 5.5195 - 0.586 * 0.8333 = 5.0311862
$$


# Problem 2

In [9]:
class CallOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price


In [10]:
class AnalyticEngine:

    def __init__(self):
        pass

    def BSpriceCall(self, dynamics, contract):

        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        return np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))

    def IV(self, dynamics, contract):

        if contract.price is None:
            raise ValueError('Contract price must be given')

        df = np.exp(-dynamics.r*contract.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-contract.K)*df])
        C = contract.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan

        dytry = copy(dynamics)
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        sigma_try = 0.2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)>C:
            sigma_try /= 2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)<C:
            sigma_try *= 2
        hi = sigma_try
        lo = hi/2

        def objective(sigma):
            dytry = copy(dynamics)
            dytry.sigma = sigma
            return self.BSpriceCall(dytry, contract) - C
        impliedVolatility = brentq(objective, a=lo, b=hi)    
        return impliedVolatility


In [11]:

hw1analytic = AnalyticEngine()
dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract2 = CallOption(K=100, T=0.5)
hw1analytic.BSpriceCall(dynamics2,contract2)

11.246291601828489

In [12]:

contract2.price = 12
hw1analytic.IV(dynamics2,contract2)    

0.4001327809210663

In [20]:

contract2.price = 11.25
hw1analytic.IV(dynamics2,contract2) 

0.4001327809210663

In [13]:
hw1analytic_sub = AnalyticEngine()
dynamics3 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract3 = CallOption(K=100, T=1)
hw1analytic_sub.BSpriceCall(dynamics3,contract3)

15.851941887820608

In [14]:
contract3.price = 12 
hw1analytic.IV(dynamics3,contract3)

0.3019384309935543

The implied volatility is 0.3019 for the call with 1-year expiry. The implied volatility is 0.4001 for the call with 0.75-year expiry.

### (b)

If the implied volatility of S with expiry 0.75 is midpoint of the previous two results, $\sigma_{imp} = 0.35103$ roughly. 

Utilizing the BS formula, we can get: 
$$C = N(\frac{1}{0.35103\sqrt{0.75}}[ln(\frac{100}{100})+\frac{0.35103^2}{2}*0.75])*100 \\- N(\frac{1}{0.35103\sqrt{0.75}}[ln(\frac{100}{100})+\frac{0.35103^2}{2}*0.75] - 0.35103*\sqrt{0.75})*100 \\ = N(0.15200) *100 - N(-0.15200)*100 = 0.5604 *100 - 0.4396*100 = 12.08$$


### (c)

At t = 0, short one unit of Call with 0.75 expirary and long one Call with 1 expirary. 

Mathematical Notation:
$$V(0) = - C_{0.75}+ C_{1} = -12.08 + 12 = -0.08 < 0 \\ P(V_0) < 0 = 1$$

Then we can list all the possibility down below: 

V(T) | $S_{0.75} \geq$ 100 | $S_{0.75} < $ 100 
------|---------------------|----------
$S_{1} \geq$ 100 | $S_1-100 > 0$ | $100 - S_1$ > 0 
$S_{1} <$ 100  | Short stock at 0.75, get 100. At t = 1, do not exercise the call, buy a stock and recover the short. Get $-S_1 - S_0.75 + 200 > 0$ | 0

$P(V_1 \geq 0) = 1$
